In [6]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()
spark

In [3]:
df_green = spark.read.parquet('../datasets/pq/green/*/*')
df_green.show(5)

+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|VendorID|lpep_pickup_datetime|lpep_dropoff_datetime|store_and_fwd_flag|RatecodeID|PULocationID|DOLocationID|passenger_count|trip_distance|fare_amount|extra|mta_tax|tip_amount|tolls_amount|ehail_fee|improvement_surcharge|total_amount|payment_type|trip_type|congestion_surcharge|
+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|       2| 2020-01-11 04:05:54|  2020-01-11 04:13:49|                 N|       1.0|         129|         129|            1.0|         0.81|        6.5|  0.5|    0.

In [7]:
df_green.createOrReplaceTempView('green')

In [24]:
# Perfrom groupby aggregation to obtain analytical insights
df_green_revenue = spark.sql("""
SELECT 
    date_trunc('hour', lpep_pickup_datetime) AS hour, 
    PULocationID AS zone,
    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    green
WHERE
    lpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
-- ORDER BY
--     1, 2
""")

df_green_revenue.show()

+-------------------+----+------------------+--------------+
|               hour|zone|            amount|number_records|
+-------------------+----+------------------+--------------+
|2020-01-17 21:00:00|  42| 446.7500000000002|            35|
|2020-01-16 07:00:00|  82| 78.36999999999999|             7|
|2020-01-26 12:00:00| 181|120.53000000000002|             7|
|2020-01-08 04:00:00| 116|             48.45|             4|
|2020-01-18 15:00:00|   7|307.06000000000006|            26|
|2020-01-16 18:00:00| 193|             60.22|             7|
|2020-01-10 14:00:00|  74| 782.1399999999995|            65|
|2020-01-14 15:00:00| 179|             98.72|             4|
|2020-01-13 07:00:00|   7|139.98000000000002|             6|
|2020-01-12 08:00:00|  74|            522.91|            26|
|2020-01-17 14:00:00| 129|321.17000000000013|            22|
|2020-01-28 19:00:00|  66| 533.9200000000001|            24|
|2020-01-05 13:00:00| 173| 94.00999999999999|             4|
|2020-01-02 17:00:00|  5

In [25]:
df_green_revenue.rdd.getNumPartitions()

18

In [28]:
# Store analytical insights as file
df_green_revenue \
    .repartition(20) \
    .write \
    .parquet("../datasets/report/revenue/green", mode='overwrite')

25/04/02 19:27:05 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
25/04/02 19:27:05 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
25/04/02 19:27:05 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 76.00% for 10 writers
25/04/02 19:27:05 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 69.09% for 11 writers
25/04/02 19:27:05 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 63.33% for 12 writers
25/04/02 19:27:05 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 58.46% for 13 writers
25/04/02 19:27:05 WARN MemoryManager: Total allocation exceeds 95.

In [31]:
df_yellow = spark.read.parquet('../datasets/pq/yellow/*/*')
df_yellow.createOrReplaceTempView('yellow')

In [32]:
df_yellow_revenue = spark.sql("""
SELECT 
    date_trunc('hour', tpep_pickup_datetime) AS hour, 
    PULocationID AS zone,
    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    yellow
WHERE
    tpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
""")

df_yellow_revenue.show()

25/04/02 19:30:38 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/04/02 19:30:38 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


+-------------------+----+------------------+--------------+
|               hour|zone|            amount|number_records|
+-------------------+----+------------------+--------------+
|2020-01-26 16:00:00| 142| 5927.010000000004|           383|
|2020-01-06 10:00:00| 236| 8045.710000000017|           568|
|2020-01-06 16:00:00| 138|17057.700000000008|           379|
|2020-01-16 12:00:00|  48| 3903.909999999998|           240|
|2020-01-21 15:00:00| 209|            703.93|            23|
|2020-01-23 10:00:00| 263|3526.0399999999972|           243|
|2020-01-07 07:00:00| 249|2012.3399999999988|           137|
|2020-01-28 13:00:00| 238|3496.9699999999984|           239|
|2020-01-17 08:00:00|  50| 2093.599999999999|           118|
|2020-01-13 09:00:00| 249|3403.5899999999997|           224|
|2020-01-26 19:00:00| 132| 26727.00999999997|           531|
|2020-01-20 14:00:00| 262| 2293.789999999999|           144|
|2020-01-04 09:00:00| 239| 3085.389999999997|           232|
|2020-01-24 07:00:00| 23

In [33]:
df_yellow_revenue \
    .repartition(20) \
    .write.parquet('../datasets/report/revenue/yellow', mode='overwrite')

25/04/02 19:30:52 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/04/02 19:30:53 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/04/02 19:30:55 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
25/04/02 19:30:55 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
25/04/02 19:30:55 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 76.00% for 10 writers
25/04/02 19:30:55 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 69.09% for 11 writers
25/04/02 19:30:55 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 63.33% for

In [34]:
# Loading reports data

df_green_revenue = spark.read.parquet('../datasets/report/revenue/green')
df_yellow_revenue = spark.read.parquet('../datasets/report/revenue/yellow')

In [ ]:
# Renaming columns before join

df_green_revenue_tmp = df_green_revenue \
    .withColumnRenamed('amount', 'green_amount') \
    .withColumnRenamed('number_records', 'green_number_records')

df_yellow_revenue_tmp = df_yellow_revenue \
    .withColumnRenamed('amount', 'yellow_amount') \
    .withColumnRenamed('number_records', 'yellow_number_records')

In [ ]:
# Performing join between yellow and green on hour and zone

df_join = df_green_revenue_tmp.join(df_yellow_revenue_tmp, on=['hour', 'zone'], how='outer')
df_join.show()

+-------------------+----+------------------+--------------------+------------------+---------------------+
|               hour|zone|      green_amount|green_number_records|     yellow_amount|yellow_number_records|
+-------------------+----+------------------+--------------------+------------------+---------------------+
|2020-01-01 00:00:00|  10|              null|                null|             42.41|                    2|
|2020-01-01 00:00:00|  43|            107.52|                   6| 6539.510000000011|                  390|
|2020-01-01 00:00:00|  52|             83.33|                   4|              49.8|                    2|
|2020-01-01 00:00:00|  70|54.900000000000006|                   3|               9.3|                    1|
|2020-01-01 00:00:00|  88|              null|                null| 823.8000000000001|                   36|
|2020-01-01 00:00:00| 140|              null|                null| 2268.909999999998|                  138|
|2020-01-01 00:00:00| 179|  

In [39]:
# Number of Partitions assigned to df_join
df_join.rdd.getNumPartitions()

21

In [40]:
# Saving the table as a parquet file
df_join \
    .write \
    .parquet("../datasets/report/revenue/total", mode='overwrite')

25/04/02 19:36:33 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
25/04/02 19:36:33 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
25/04/02 19:36:33 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 76.00% for 10 writers
25/04/02 19:36:33 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 69.09% for 11 writers
25/04/02 19:36:33 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 63.33% for 12 writers
25/04/02 19:36:33 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 58.46% for 13 writers
25/04/02 19:36:33 WARN MemoryManager: Total allocation exceeds 95.

In [42]:
# Read the saved parquet file in folder 'total'
df_join = spark.read.parquet('../datasets/report/revenue/total/')
df_join.show()

+-------------------+----+------------------+--------------------+------------------+---------------------+
|               hour|zone|      green_amount|green_number_records|     yellow_amount|yellow_number_records|
+-------------------+----+------------------+--------------------+------------------+---------------------+
|2020-01-01 00:00:00|  22|              15.8|                   1|              null|                 null|
|2020-01-01 00:00:00|  38| 98.78999999999999|                   2|              null|                 null|
|2020-01-01 00:00:00|  72|             67.53|                   2|              null|                 null|
|2020-01-01 00:00:00| 114|              null|                null| 6256.430000000006|                  333|
|2020-01-01 00:00:00| 116|328.76000000000005|                  18| 561.5100000000001|                   34|
|2020-01-01 00:00:00| 132|              null|                null| 6786.390000000004|                  139|
|2020-01-01 00:00:00| 136|  